In [1]:
%%capture
! pip install ml-collections
! pip3 install pickle5
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/WhenPigsFlyContext/baselines/gbvs

In [2]:
import sys
import cv2
from google.colab.patches import cv2_imshow
import time
from matplotlib import pyplot as plt
from tqdm import tqdm, trange
import numpy as np
import pandas as pd
import pickle
import pickle5 as pickle5
import random
from random import sample
import copy

import os
import shutil
from PIL import Image, ImageDraw

import torch
from torch.utils.data import Dataset
from torchvision.transforms.functional import to_tensor, normalize
from torchvision import transforms

from saliency_models import gbvs, ittikochneibur
from SCEGRAM.SCEGRAM import SCEGRAM
from utils import *
sys.path.append("..")

In [3]:
context_dir = "../../SCEGRAM/01scenes/01object_present"
# target_dir = "../../SCEGRAM/02objects"
target_dir = "../../SCEGRAM/invariant_objects"
info_dir = "../../SCEGRAM/SCEGRAM_Database_scenes_objects.xlsx"
context_size, target_size = (224, 224), (224, 224)
dataset = SCEGRAM(info_dir, context_dir, target_dir, context_size, target_size, is_transform=False)

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [4]:
with open("../IVSN/[SCEGRAM]bin_idxs.pkl", "rb") as tf:
    bin_info = pickle5.load(tf) 

In [5]:
size, img_size = 48, (320, 512)
gbvs_CON_0_25, gbvs_CON_25_50 = [], []
gbvs_INCON_0_25, gbvs_INCON_25_50 = [], []
scanpath, gbvs_attention_map = {}, {}
selected_imgs = bin_info['con_(0, 25]'].tolist() + bin_info['con_(25, 50]'].tolist() + bin_info['incon_(0, 25]'].tolist() + bin_info['incon_(25, 50]'].tolist()

for id in trange(0, len(dataset)):
    if id not in selected_imgs:
        continue

    img, _, bbox_relative, category = dataset[id]
    # get the target bounding box
    tg_loc = bbox_cordinates(bbox_relative, img_size[1], img_size[0])

    # compute the saliency map
    saliency_map_gbvs = gbvs.compute_saliency(img)

    saliency_map_gbvs_tensor = torch.from_numpy(saliency_map_gbvs).detach()
    saliency_map_gbvs_tensor = transforms.Resize(img_size)(saliency_map_gbvs_tensor.unsqueeze(0))

    # save the attention map
    gbvs_attention_map[id] = copy.deepcopy(saliency_map_gbvs_tensor)

    search_num, path = searchProcesswithPath(tg_loc, saliency_map_gbvs_tensor, img_size, size)
    scanpath[id] = path

    if id in bin_info['con_(0, 25]'].tolist():
        gbvs_CON_0_25.append(search_num)
    elif id in bin_info['con_(25, 50]'].tolist():
        gbvs_CON_25_50.append(search_num)

    elif id in bin_info['incon_(0, 25]'].tolist():
        gbvs_INCON_0_25.append(search_num)
    elif id in bin_info['incon_(25, 50]'].tolist():
        gbvs_INCON_25_50.append(search_num)

    print("search times_{}: ".format(id), search_num)

gbvs_CON_res = gbvs_CON_0_25 + gbvs_CON_25_50
gbvs_INCON_res = gbvs_INCON_0_25 + gbvs_INCON_25_50
gbvs_res = gbvs_CON_res + gbvs_INCON_res 

  0%|          | 1/372 [00:09<59:15,  9.58s/it]

search times_0:  25


  1%|          | 2/372 [00:16<49:16,  7.99s/it]

search times_1:  25


  1%|          | 3/372 [00:23<47:26,  7.71s/it]

...

search times_364:  111


 99%|█████████▉| 370/372 [19:08<00:03,  1.98s/it]

search times_369:  3


100%|██████████| 372/372 [19:13<00:00,  3.10s/it]

search times_371:  40


In [6]:
np.mean(gbvs_res), np.mean(gbvs_CON_res), np.mean(gbvs_INCON_res)

(18.32085561497326, 10.78125, 19.87741935483871)

In [8]:
def sampleIncon(incon_bin_result, con_bin_result, times):
    sample_times = times
    nums = len(con_bin_result)
    res = np.array([0.0] * 25)
    
    for id in range(sample_times):
        temp = sample(incon_bin_result, nums)
        temp_accu = model_performance(temp, len(temp))
        res += np.array(temp_accu[:25])

    return (res/sample_times).tolist()

def balanced_accu(res_con, res_incon):
    res = []
    for i in range(25):
        res.append((res_con[i]+res_incon[i])/2)

    return res

In [ ]:
times = 100
gbvs_CON_accu = model_performance(gbvs_CON_res, len(gbvs_CON_res))
gbvs_INCON_accu = sampleIncon(gbvs_INCON_res, gbvs_CON_res, times)
gbvs_accu = balanced_accu(gbvs_CON_accu, gbvs_INCON_accu)
gbvs_accu[:11], gbvs_CON_accu[:11], gbvs_INCON_accu[:11]

In [10]:
gbvs_SCEGRAM_res = {}
gbvs_SCEGRAM_res['combined_accu'] = gbvs_accu
gbvs_SCEGRAM_res['con_accu'] = gbvs_CON_accu
gbvs_SCEGRAM_res['incon_accu'] = gbvs_INCON_accu
gbvs_SCEGRAM_res['con_[0,25)'] = gbvs_CON_0_25
gbvs_SCEGRAM_res['con_[25,50)'] = gbvs_CON_25_50
gbvs_SCEGRAM_res['incon_[0,25)'] = gbvs_INCON_0_25
gbvs_SCEGRAM_res['incon_[25,50)'] = gbvs_INCON_25_50
gbvs_SCEGRAM_res['scanpath'] = scanpath
gbvs_SCEGRAM_res['attention_map'] = gbvs_attention_map

In [11]:
with open("../results/SCEGRAM/SCEGRAM(invariant_bin1_2)_gbvs_res.pkl", "wb") as tf:
    pickle.dump(gbvs_SCEGRAM_res, tf)